In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import mlflow
import optax
import polars as pl
from flax import nnx
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

from nn_trainer.flax.evaluator import MeanSquaredErrorEvaluator
from nn_trainer.flax.loader import MiniBatchLoader
from nn_trainer.flax.loss_fn import mean_squared_error
from nn_trainer.flax.model import RegressionMLP
from nn_trainer.flax.trainer import Trainer

jax.default_backend()

'cpu'

In [3]:
housing = fetch_california_housing()
train_dataset_df, valid_dataset_df = train_test_split(
    pl.from_numpy(data=housing.data, schema=housing.feature_names).with_columns(  # type: ignore
        pl.Series(housing.target).alias(housing.target_names[0])  # type: ignore
    ),
    test_size=0.2,
    random_state=0,
)
train_X_df, train_y_df, valid_X_df, valid_y_df = (
    train_dataset_df.drop(housing.target_names[0]),  # type: ignore
    train_dataset_df.select(housing.target_names[0]),  # type: ignore
    valid_dataset_df.drop(housing.target_names[0]),  # type: ignore
    valid_dataset_df.select(housing.target_names[0]),  # type: ignore
)

In [4]:
model = RegressionMLP(train_dataset_df.shape[1] - 1, [10, 10], 1, nnx.Rngs(0))
train_loader = MiniBatchLoader(X_df=train_X_df, y_df=train_y_df, batch_size=512, seed=0)
valid_evaluator = MeanSquaredErrorEvaluator(X_df=valid_X_df, y_df=valid_y_df)

### Training

In [5]:
%%time

mlflow.set_tracking_uri(uri="http://localhost:8080")
mlflow.set_experiment("REGRESSION")

with mlflow.start_run() as run:
    mlflow.log_param("my", "param")

    trainer = Trainer(
        model=model,
        optimizer=optax.adam(learning_rate=0.001),
        train_loader=train_loader,
        loss_fn=mean_squared_error,
        valid_evaluator=valid_evaluator,
        early_stopping_patience=10,
        epoch_num=512,
        active_run=run,
    )
    trainer = trainer.fit()

[VALID 000]: loss=8120.650390625, metrics={'MSE': 8120.650390625}


[TRAIN 001]: 100%|██████████| 32/32 [00:00<00:00, 54.47it/s, batch_loss=51.6]  


[VALID 001]: loss=33.786903381347656, metrics={'MSE': 33.786903381347656}


[TRAIN 002]: 100%|██████████| 32/32 [00:00<00:00, 296.51it/s, batch_loss=15.7]


[VALID 002]: loss=19.98079490661621, metrics={'MSE': 19.98079490661621}


...

[VALID 096]: loss=0.8970376253128052, metrics={'MSE': 0.8970376253128052}


[TRAIN 097]: 100%|██████████| 32/32 [00:00<00:00, 410.87it/s, batch_loss=0.777]


[VALID 097]: loss=1.2914140224456787, metrics={'MSE': 1.2914140224456787}


[TRAIN 098]: 100%|██████████| 32/32 [00:00<00:00, 259.32it/s, batch_loss=0.904]


[VALID 098]: loss=0.8974832892417908, metrics={'MSE': 0.8974832892417908}
🏃 View run delightful-ray-736 at: http://localhost:8080/#/experiments/663538387065204330/runs/703148feb0204591bbb6ad6ebe51171f
🧪 View experiment at: http://localhost:8080/#/experiments/663538387065204330
CPU times: user 10.4 s, sys: 1 s, total: 11.4 s
Wall time: 11.2 s


### Inference

In [6]:
valid_loader = MiniBatchLoader(X_df=train_X_df, y_df=train_y_df, batch_size=512, seed=0)
valid_loader.setup_epoch()
for Xs, y in valid_loader:
    break
trainer.best_model(*Xs)[:10]

Array([[2.151601  ],
       [2.0596173 ],
       [0.92314255],
       [2.4030411 ],
       [2.4513104 ],
       [2.3431027 ],
       [1.8763744 ],
       [1.5203944 ],
       [3.1518376 ],
       [2.1497128 ]], dtype=float32)